In [1]:
!pip install selenium
import time
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import requests 
import pandas as pd
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')
import re
import sys

In [5]:
name = input('Введите ФИО чиновника, который Вас интересует - обязательно в таком порядке: ')

Введите ФИО чиновника, который Вас интересует - обязательно в таком порядке: Никитин Глеб Сергеевич


In [6]:
def declarator():
    
    br = wb.Chrome(r'C:\Users\vladi\Documents\chromedriver_win32\chromedriver.exe')
    br.get('https://google.com')
    x = name + ' ' + 'declarator.org/person -charts '
    search = br.find_element_by_name('q')
    search.send_keys(x)
    search.send_keys(Keys.RETURN)
        
    try:
        br.find_element_by_tag_name("cite").click() 
    except:
        print('Пожалуйста, перезапустите программу.')
        return
        
    try:
        link = br.find_elements_by_class_name('show_person_details')
    except:
        print('К сожалению, мы не смогли найти деклараций о доходах данного чиновника. Попробуйте найти декларации другого чиновника.')
        return
    
    for x in range(0,len(link)):
        link[x].click()
    time.sleep(5)
    a = br.find_elements_by_class_name('section_detailed')
    if a:
        c = []
        for x in range(0,len(a)):
            c.append(a[x].text.replace('\n', ' ').split('Скрыть подробности'))
                
        
    try:
        response = requests.get(br.current_url)
        soup = BeautifulSoup(response.text)
        data_per_year = soup.select('body > div.page_body.person-new > table')
        df = pd.read_html(str(data_per_year))[0]
    except:
        print('К сожалению, мы не смогли найти деклараций о доходах данного чиновника. Попробуйте найти декларации другого чиновника.')
        return
        
    df1 = df.dropna(how='all', axis=1)
    df2 = df1.iloc[::2]
    df2.index = range(len(df2))
        
    df2['Дополнительно'] = c
    df2['Декларация, статус, учреждение'] = df2['Декларация, статус, учреждение'].map(lambda x: str(x)[:-20])
    df2['Доход, руб.'] = df2['Доход, руб.'].map(lambda x: str(x)[:-5])
    df2['Доход, руб.'] = df2['Доход, руб.'].str.replace(' ', '')
    df2['Недвижимость, м2'] = df2['Недвижимость, м2'].map(lambda x: str(x)[:-3])
    df2['Недвижимость, м2'] = df2['Недвижимость, м2'].str.replace(' ', '')
    df2['Транспорт, шт.'] = df2['Транспорт, шт.'].map(lambda x: str(x)[:-3])
    df2['Транспорт, шт.'] = df2['Транспорт, шт.'].str.replace(' ', '')
    df2['Дополнительно'] = df2['Дополнительно'].map(lambda x: str(x)[:-2])
    df2['Дополнительно'] = df2['Дополнительно'].map(lambda x: str(x)[6:])
    df2['Дополнительно'] = df2['Дополнительно'].str.replace('  ', '')
        
    df2["Доход, руб."] = pd.to_numeric(df2["Доход, руб."])
    df2["Недвижимость, м2"] = pd.to_numeric(df2["Недвижимость, м2"])
    df2["Транспорт, шт."] = pd.to_numeric(df2["Транспорт, шт."])

    df2["Дополнительно"] = df2["Дополнительно"].str.strip()
    df2.style.set_properties(**{'text-align': 'left'})
    f = input('Хотите ли вы скачать данные в excel-формате? Ответьте да или нет: ')
    if f == 'да': 
        df2.to_excel("df_dec4.xlsx")
        return df2
           
    else:
        return df2

    
declarator()

Хотите ли вы скачать данные в excel-формате? Ответьте да или нет: да


,"Декларация, статус, учреждение","Доход, руб.","Недвижимость, м2","Транспорт, шт.",Дополнительно
0,Антикоррупционная декларация 2019 Губернатор ...,4869247,2135,7,"Доход 4 869 246,76 руб. Супруг(а): 3 241 168,7..."
1,Антикоррупционная декларация 2018 Губернатор ...,4692512,2027,4,"Доход 4 692 512,11 руб. Супруг(а): 5 885 506,7..."
2,Антикоррупционная декларация 2017 Временно ис...,15737696,2027,4,"Доход 15 737 696,32 руб. (в т.ч. доход, получе..."
3,Антикоррупционная декларация 2016 Первый заме...,8323749,5690,4,"Доход 8 323 748,80 руб. Супруг(а): 25 669 281,..."
4,Антикоррупционная декларация 2016 Временно ис...,8323749,5833,4,"Доход 8 323 748,80 руб. Супруг(а): 25 669 281,..."
5,Антикоррупционная декларация 2015 Первый заме...,6789479,1981,3,"Доход 6 789 479,06 руб. Супруг(а): 17 207 848,..."
6,Антикоррупционная декларация 2014 Первый заме...,7250349,1877,3,"Доход 7 250 349,02 руб. Супруг(а): 12 405 322,..."
7,Антикоррупционная декларация 2013 Первый заме...,6336395,1946,3,"Доход 6 336 394,55 руб. Супруг(а): 12 300 542,..."


In [8]:
def telegram():
    br = wb.Chrome(r'C:\Users\vladi\Documents\chromedriver_win32\chromedriver.exe')
    time.sleep(1)
    br.get('https://web.telegram.org/#/im?p=@Gubery')
    time.sleep(5)
    try:
        br.find_element_by_css_selector('body > div.page_wrap > div > div.login_page > div.login_form_wrap > form > div.login_phone_groups_wrap.clearfix > div.md-input-group.login_phone_num_input_group.md-input-animated > input').click()
        time.sleep(3)
        number = input('Введите номер телефона слитно и без +7: ')
        br.find_element_by_css_selector('body > div.page_wrap > div > div.login_page > div.login_form_wrap > form > div.login_phone_groups_wrap.clearfix > div.md-input-group.login_phone_num_input_group.md-input-animated > input').send_keys(number)
        time.sleep(2)
        br.find_element_by_css_selector('body > div.page_wrap > div > div.login_page > div.login_head_wrap.clearfix > div > a').click()
        time.sleep(2)
        br.find_element_by_css_selector('body > div.modal.fade.confirm_modal_window.in > div.modal-dialog > div > div > div.md_simple_modal_footer > button.btn.btn-md.btn-md-primary').click()
        code = input('Введите отправленный код: ')
        time.sleep(1)
        br.find_element_by_css_selector('body > div.page_wrap > div > div.login_page > div.login_form_wrap > form > div.md-input-group.md-input-group-centered.md-input-animated > input').send_keys(code)
        time.sleep(1)
    except:
        print('Пожалуйста, перезапустите программу')
        return

    df = pd.DataFrame(columns = ['Пост', 'Дата поста'])
    name_p = name.split()[0]
    time.sleep(2)
    br.get('https://web.telegram.org/#/im?p=@Gubery')
    time.sleep(3)
    br.find_element_by_css_selector('body > div.page_wrap > div:nth-child(1) > div > div > div.tg_head_main_wrap > div > a.tg_head_btn.tg_head_peer_search_btn').click()
    time.sleep(2)
    br.find_element_by_css_selector('body > div.page_wrap > div.im_page_wrap.clearfix > div > div.im_dialogs_col_wrap.noselect > div.im_dialogs_panel > div > input').send_keys(name_p)
    time.sleep(3)
    l_news = []
    l_date = []
    b = br.find_elements_by_class_name('im_dialog')
    time.sleep(3)
    for i in range(1, len(b)):
        br.find_element_by_css_selector(f'body > div.page_wrap > div.im_page_wrap.clearfix > div > div.im_dialogs_col_wrap.noselect.im_dialogs_col_search > div.im_dialogs_col > div > div.im_dialogs_scrollable_wrap.nano-content > div.im_dialogs_messages_wrap > ul > li:nth-child({i}) > a').click()
        time.sleep(4)
        for news in br.find_elements_by_xpath('.//*[@id="ng-app"]/body/div[1]/div[2]/div/div[2]/div[3]/div/div[2]/div[1]/div/div[1]/div[2]/div[2]/div[21]/div/div/div/div[2]/div[2]'):
            l_news.append(news.text)
            time.sleep(3)
        for date in br.find_elements_by_xpath('.//*[@id="ng-app"]/body/div[1]/div[2]/div/div[2]/div[3]/div/div[2]/div[1]/div/div[1]/div[2]/div[2]/div[21]/div/div/div/div[2]/div[1]/div/span[2]'):
            l_date.append(date.text)
            time.sleep(3)
                           


    l_date = [x for x in l_date if not x.startswith(':')]


    df['Пост'] = l_news
    try:
        df['Дата поста'] = l_date
    except:
        l_date = [x for x in l_date if x != ':speak_no_evil:']
        l_news = [x for x in l_date if x != ':speak_no_evil:']
        l_date = [x for x in l_date if x != ':see_no_evil:']
        l_news = [x for x in l_date if x != ':see_no_evil:']
        l_date = [x for x in l_date if x != ':white_check_mark:']
        l_news = [x for x in l_date if x != ':white_check_mark:']
        l_news = [x for x in l_date if x != ':heavy_check_mark:']
        l_date = [x for x in l_date if x != ':heavy_check_mark:']
        l_date = [x for x in l_date if x != '']
        l_news = [x for x in l_date if x != '']
        try:
             df['Дата поста'] = l_date
        except:
            df1 = pd.DataFrame(columns = ['Пост'])
            df2 = pd.DataFrame(columns = ['Дата поста'])
            df1['Пост'] = l_news
            df2['Дата поста'] = l_date
            try:
                df = pandas.concat([df1, df2], axis=1) 
            except:
                print('Пожалуйста, перезапустите программу')
                return

    f_n = input('Хотите ли вы скачать данные в excel-формате? Ответьте да или нет: ')
    if f_n == 'да': 
        df.to_excel("df_news4.xlsx")
        return df   
    else:
        return df
    
telegram()

Введите номер телефона слитно и без +7: 9038428118
Введите отправленный код: 65775
Хотите ли вы скачать данные в excel-формате? Ответьте да или нет: да


,Пост,Дата поста
0,"Продолжается акция ""Новый год в каждый дом""! П...","Dec 24, 2020 11:10:39 AM"
1,Сегодня в Москве президент России Владимир Пут...,"Dec 23, 2020 4:28:32 PM"
2,Биржа губернаторов :bar_chart:\n23/12/2020\n\n...,"Dec 23, 2020 12:01:00 AM"
3,Биржа губернаторов :bar_chart:\n23/12/2020\n\n...,"Dec 23, 2020 12:01:00 AM"
4,Фактор коронавируса в контексте заболевания 29...,"Dec 21, 2020 6:03:28 PM"
5,Фактор коронавируса в контексте заболевания 29...,"Dec 21, 2020 6:03:28 PM"
6,Фактор коронавируса в контексте заболевания 29...,"Dec 21, 2020 6:03:28 PM"
7,Реализацию национальных проектов в Нижегородск...,"Dec 18, 2020 6:07:18 PM"
8,Биржа губернаторов :bar_chart:\n18/12/2020\n\n...,"Dec 18, 2020 12:01:00 AM"
9,Губернатор Нижегородской области Глеб Никитин ...,"Dec 17, 2020 2:28:23 PM"
